<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Graded_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275 - Advanced Supply Chain Systems Planning and Network Design
# **Graded Assignment 2**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 33.9 MB/s eta 0:00:00


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from folium import Choropleth         # Import Choropleth from folium for creating choropleth maps
from geopy.distance import geodesic   # For calculating geodesic distances between two points
import geopandas as gpd               # Import GeoPandas for working with geospatial data and extending pandas functionality to handle geometry

## **Helper functions**

### **Plotting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi')

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Plotting geographies - choropleth map**

In [ ]:
# Defining a function for creating a choropleth map

def plot_geographies_choropleth(map,                  # Folium map object to plot the geographies on
                                geographies,          # Dictionary of objects containing geographical data
                                attribute_to_plot,    # Object attribute used to create the map
                                fill_color="YlGn"):   # Color used


    # Create a pandas DataFrame from the geographies for easier plotting
    # Note: getattr() function allows to retrieve an attribute by name; for example: getattr(demand_zone_1, 'selling_price') provides the selling price of the demand_zone_1

    data = pd.DataFrame({
        'ID': [geo.ID for geo in geographies.values()],
        attribute_to_plot: [getattr(geo, attribute_to_plot) for geo in geographies.values()],
        'geometry': [geo.geometry for geo in geographies.values()],
    })


    # Convert the DataFrame to GeoJSON format
    # GeoJSON format is used to represent geographic data in a JSON-like structure. It includes the type of geometry (e.g., Polygon, Point) and the coordinates.

    geo_json_data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": geo.geometry.__geo_interface__,  # Convert the geometry to GeoJSON format using the __geo_interface__ attribute of the geometry object
                "properties": {"ID": geo.ID, attribute_to_plot: getattr(geo, attribute_to_plot)}
            }
            for geo in geographies.values()
        ]
    }

    # Add a Choropleth layer to the map
    # Choropleth is a type of map where areas are shaded based on the value of a variable, in this case, the 'attribute_to_plot'.

    Choropleth(
        geo_data=geo_json_data,  # Use the GeoJSON data for geometry
        name="choropleth",
        data=data,
        columns=["ID", attribute_to_plot],
        key_on="feature.properties.ID",
        fill_color=fill_color,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=attribute_to_plot,
    ).add_to(map)


### **Plotting geographies - categorical map**

In [ ]:
def plot_geographies_by_category(map,                   # Folium map object to plot the geographies on
                                 geographies,           # Dictionary of objects containing geographical data
                                 attribute_to_plot,     # Object attribute used to create the map
                                 category_colors):      # Color used

    # Create a pandas DataFrame from the geographies for easier plotting
    data = pd.DataFrame({
        'ID': [geo.ID for geo in geographies.values()],
        attribute_to_plot: [getattr(geo, attribute_to_plot) for geo in geographies.values()],
        'geometry': [geo.geometry for geo in geographies.values()],
    })

    # Convert the DataFrame to GeoJSON format
    geo_json_data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": geo.geometry.__geo_interface__,  # Convert geometry to GeoJSON format
                "properties": {"ID": geo.ID, attribute_to_plot: getattr(geo, attribute_to_plot)}
            }
            for geo in geographies.values()
        ]
    }

    # Add polygons to the map and color them based on the specified category color mapping
    for _, row in data.iterrows():
        color = category_colors.get(row[attribute_to_plot], "gray")  # Default to 'gray' if category not in category_colors
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 0.5,
                'fillOpacity': 0.7,
                'lineOpacity': 0.2,
            },
            tooltip=f"ID: {row['ID']}, {attribute_to_plot}: {row[attribute_to_plot]}"
        ).add_to(map)

    # Add a legend for the categories
    legend_html = """
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 150px; height: 150px;
                background-color: white; z-index:9999; font-size:14px;">
    <h4>Legend</h4>
    <ul style="list-style-type: none; padding: 0;">"""
    for cat, color in category_colors.items():
        legend_html += f"<li><span style='background-color:{color}; padding:5px;'></span> {cat}</li>"
    legend_html += "</ul></div>"

    map.get_root().html.add_child(folium.Element(legend_html))


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing containing demand zone data
demand_zones_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/graded_assignment_2_service_level/demand_zones.json'

# Loading demand zone data into a pandas DataFrame
demand_zones_df = gpd.read_file(demand_zones_file)

# # Keeping only demand zones in certain states
# demand_zones_df = demand_zones_df.loc[demand_zones_df.state.isin(['New Jersey', 'New York', 'Pennsylvania', 'Connecticut'])]

# Displaying the first few rows of the DataFrame to verify the data
demand_zones_df.head()

,ID,county,state,lon,lat,max_demand,selling_price,geometry
0,36093,Schenectady County,New York,-74.052783,42.816688,641,239,"POLYGON ((-74.26331 42.79653, -74.08388 42.897..."
1,36051,Livingston County,New York,-77.779662,42.725963,247,234,"POLYGON ((-78.06062 42.53281, -77.95633 42.667..."
2,36037,Genesee County,New York,-78.198036,43.000860,231,246,"POLYGON ((-78.4655 43.12862, -78.02723 43.1320..."
3,36009,Cattaraugus County,New York,-78.679188,42.247842,306,245,"POLYGON ((-79.06078 42.53785, -78.9917 42.5292..."
4,42117,Tioga County,Pennsylvania,-77.254438,41.772558,165,233,"POLYGON ((-77.61002 41.99915, -77.00764 42.000..."


In [ ]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/graded_assignment_2_service_level/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()

,ID,name,lat,lon,fixed_cost,capacity,landed_cost
0,w1,Albany,42.6664,-73.7987,1000000,40000,100
1,w2,Jersey City,40.7184,-74.0686,1500000,50000,105
2,w3,Harrisburg,40.2752,-76.8843,1000000,45000,110


#### Definition of Classes

In [ ]:
# Class representing a DemandZone object

class DemandZone():
    def __init__(self, ID, lat, lon, max_demand, selling_price, geometry):
        self.ID = ID                          # DemandZone's ID
        self.lat = lat                        # DemandZone's latitude
        self.lon = lon                        # DemandZone's longitude
        self.max_demand = max_demand          # DemandZone's maximum demand
        self.selling_price = selling_price    # DemandZone's population
        self.geometry = geometry              # DemandZone's geometry

        self.demand = dict()                  # DemandZone's demand per service level


In [ ]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, fixed_cost, capacity, landed_cost):
        self.ID = ID                              # Warehouse's ID
        self.name = name                          # Warehouse's name
        self.lat = lat                            # Warehouse's latitude
        self.lon = lon                            # Warehouse's longitude
        self.fixed_cost = fixed_cost              # Warehouse's fixed cost
        self.capacity = capacity                  # Warehouse's capacity
        self.landed_cost = landed_cost            # Warehouse's landed cost

        self.active = True                        # Initializing node as active


#### Creating node objects

In [ ]:
# Initializing an empty dictionary to store node objects
nodes = dict()

In [ ]:
# Creating a dictionary of demand zone objects

demand_zones = dict()
for i, row in demand_zones_df.iterrows():
    demand_zones[row['ID']] = DemandZone(ID = row['ID'],                              # DemandZone's ID
                                         lat = row['lat'],                            # DemandZone's latitude
                                         lon = row['lon'],                            # DemandZone's longitude
                                         max_demand = row['max_demand'],              # DemandZone's maximum demand
                                         selling_price = row['selling_price'],        # DemandZone's selling price
                                         geometry = row['geometry'])                  # DemandZone's geometry (geospatial data)

# Merging the demand zones dictionary into the existing nodes dictionary
nodes = {**nodes, **demand_zones}

In [ ]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],                             # Warehouse's ID
                                    name = row['name'],                           # Warehouse's name
                                    lat = row['lat'],                             # Warehouse's latitude
                                    lon = row['lon'],                             # Warehouse's longitude
                                    fixed_cost = row['fixed_cost'],               # Warehouse's fixed cost
                                    capacity = row['capacity'],                   # Warehouse's capacity
                                    landed_cost = row['landed_cost'])             # Warehouse's landed cost

# Merging the warehouses dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

#### Visualizing node objects

In [ ]:
# Create a new map
map = folium.Map([42, -75.0], zoom_start=7)

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')

# Plot demand zones on a map
plot_geographies_choropleth(map=map, geographies=demand_zones, attribute_to_plot='max_demand', fill_color = 'YlGnBu')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Service levels**

In [ ]:
# Defining a list of service levels to consider in the model
service_levels = ['slow', 'fast']

### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary containing distances between warehouses and demand_zones
distances = dict()
for w, warehouse in warehouses.items():
  for d, demand_zone in demand_zones.items():
      distances[w, d] = compute_geodesic_distance(origin = warehouse, destination = demand_zone, unit = 'km')

#### Arc costs

***❗Task 1: Computing unit arc costs***

Create a dictionary of unit transportation costs, `unit_cost`, that accounts for the origin, destination, and service level. The keys should correspond to the origin, destination, and service level. Then, note down the average unit arc cost by observing the output of the print statement included in this code cell.

In [ ]:
cost_unit_km_slow = 0.85      # Cost per unit per kilometer from warehouse to demand zones with a slow service
cost_unit_km_fast = 1.25      # Cost per unit per kilometer from warehouse to demand zones with a fast service

# Creating a dictionary containing unit costs between between warehouses and demand zones
unit_cost = dict()

##### Task 1 : Your code here





# Printing the average unit arc cost
if len(unit_cost) > 0 :
  average_arc_unit_cost = sum(unit_cost.values()) / len(unit_cost)
  print ('The average unit arcs cost is', round(average_arc_unit_cost,1))


### **Processing demand**

***❗Task 2: Preprocessing demand***

For each demand zone, update the `demand` dictionary to calculate the total demand that can be captured at both the slow and fast service levels. Then, run the code cell and check the output from the print statement, which shows the demand for the `'slow'` service level in the demand zone `'42015'`.

In [ ]:
# Establishing captured demand for each demand zone, product, service level and serving warehouse

share_demand_fast = 1.00      # The share of the total demand that we can capture from a given demand zone with the fast service level
share_demand_slow = 0.80      # The share of the total demand that we can capture from a given demand zone with the fast service level


##### Task 2 : Your code here






# Printing the demand for the slow service level for the demand zone '42015'
if 'slow' in demand_zones['42015'].demand.keys():
  print (demand_zones['42015'].demand['slow'])

## **Optimization model**

### **Creating and solving the optimization model**

***❗Task 3: Redefine the transportation cost expression***

In the following code, we indicated that `transportation_cost = 0` as a placeholder. Redefine this to account for the appropriate expression as defined in the optimization model.

***❗Task 4: Define the demand constraint***

Define a constraint that signifies that each demand zone can be served by at most one warehouse at one service level.

***❗Task 5: Define the warehouse capacity constraint***

Define a constraint that signifies that the total demand served by a warehouse cannot exceed its capacity.

In [ ]:
# Initialize the optimization model for a generalized supply chain network design (SCND) with revenue
model = grb.Model("Generalized SCND and Revenue")

# Set the parameter for the maximum allowable gap for the Mixed Integer Program (MIP)
model.setParam('MIPGap', 0.0000001)

# Define dictionaries to hold binary variables for allocation (warehouse-demand zone-service level)
# and location (warehouses)
allocation_vars = dict()
location_vars = dict()

# Initialize binary variables for warehouse allocations to demand zones at different service levels
for w, warehouse in warehouses.items():
    for d, demand_zone in demand_zones.items():
        for s in service_levels:
            allocation_vars[w, d, s] = model.addVar(vtype=grb.GRB.BINARY,
                                                    name="allocation_vars_{0}_{1}_{2}".format(w, d, s))

# Initialize binary variables to indicate if each warehouse is open
for w, warehouse in warehouses.items():
    location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                                    name="location_vars{0}".format(w))

# Calculate total revenue based on the selling price, demand, and allocation for each service level
revenue = grb.quicksum(demand_zone.selling_price * allocation_vars[w, d, s] * demand_zone.demand[s]
                       for w, warehouse in warehouses.items()
                       for d, demand_zone in demand_zones.items()
                       for s in service_levels)

# Calculate total landed cost for serving each demand zone from a warehouse at a certain service level
landed_cost = grb.quicksum(warehouse.landed_cost * allocation_vars[w, d, s] * demand_zone.demand[s]
                           for w, warehouse in warehouses.items()
                           for d, demand_zone in demand_zones.items()
                           for s in service_levels)

# Calculate total fixed cost for opening each warehouse
fixed_cost_warehouses = grb.quicksum(warehouse.fixed_cost * location_vars[w]
                                     for w, warehouse in warehouses.items())


# Calculate total distribution cost based on unit cost for each warehouse-demand zone-service level allocation

transportation_cost = 0
##### Task 3 : Your code here





# Define the profit as the objective function (total revenue minus costs)
profit = revenue - transportation_cost - fixed_cost_warehouses - landed_cost

# Constraint: Each demand zone can be served by at most one warehouse at one service level

##### Task 4 : Your code here






# # Constraint: The total demand served by a warehouse cannot exceed its capacity

##### Task 5 : Your code here






# Set the objective of the model to maximize profit
model.setObjective(profit, grb.GRB.MAXIMIZE)

# Solve the optimization model
model.optimize()


In [ ]:
# Post-processing model results

for d, demand_zone in demand_zones.items():
    # Set the captured demand share for each product and service level
    setattr(demand_zone, 'captured_demand',
            grb.quicksum(allocation_vars[w, d, s] * demand_zone.demand[s]
                         for w in warehouses for s in service_levels).getValue())

    setattr(demand_zone, 'captured_ms', demand_zone.captured_demand / demand_zone.max_demand)

    demand_zone.service_level = 'none'
    for s in service_levels:
      if grb.quicksum(allocation_vars[w, d, s] for w in warehouses).getValue() > 0:
        setattr(demand_zone, 'service_level', s)

for w, warehouse in warehouses.items():
  # Set warehouse status based on location variable
  warehouse.active = True if location_vars[w].X == 1 else False


### **Solution visualization and analysis**

In [ ]:

# Create a new map
map = folium.Map([42, -75.0], zoom_start=7)

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')

# Plot demand zones on a map
plot_geographies_choropleth(map=map, geographies=demand_zones, attribute_to_plot='captured_ms', fill_color = 'YlGnBu')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map



In [ ]:
# Create a new map
map = folium.Map([42, -75.0], zoom_start=7)

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')


category_colors = {'none': 'grey', 'slow': 'orange', 'fast': 'green'}

# Plot demand zones on a map
plot_geographies_by_category(map=map, geographies=demand_zones, attribute_to_plot='service_level', category_colors = category_colors)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map